In [ ]:
import pandas as pd
import pathlib
import cv2
from tkinter import *
import os
import numpy as np
from PIL import Image
import cv2.face 

def input_nama():  
    global data_nama
    data = pd.read_excel('Deteksi Wajah/Tabel_Deteksi_Wajah.xlsx',header=0)
    nama = data_nama.get()
    cascade_path = pathlib.Path(cv2.__file__).parent.absolute()/"data/haarcascade_frontalface_default.xml"
    clf = cv2.CascadeClassifier(str(cascade_path))
    path = 'Deteksi Wajah/Dataset'
    if len(os.listdir(path)) <= 2:
        id = 1
    else:
        Ids=[]
        imagePaths=[os.path.join(path,f) for f in os.listdir(path) if f not in ['.DS_Store', '~$aining.xml','training.xml']]
        for image in imagePaths:
            Id=int(os.path.split(image)[-1].split(".")[2])
            Ids.append(Id)
        max_id = max(Ids)
        print(max_id)
        id = max_id + 1
    camera = cv2.VideoCapture(0)
    data_baru = pd.DataFrame({'Label':id, 'Nama':nama},index=[0])
    data = pd.concat([data,data_baru], ignore_index=True)
    data.to_excel('Deteksi Wajah/Tabel_Deteksi_Wajah.xlsx',index=False)
    a = 0
    while True:
        a = a+1
        check, frame = camera.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = clf.detectMultiScale(
        gray,
        scaleFactor = 1.1,
        minNeighbors = 10,
        minSize=(30,30),
        flags = cv2.CASCADE_SCALE_IMAGE)
        for (x,y,width,height) in faces:
            cv2.imwrite("Deteksi Wajah/Dataset/User."+nama+'.'+str(id)+'.'+str(a)+'.jpg',gray[y:y+height,x:x+width])
            cv2.rectangle(frame, (x,y),(x+width, y+height),(255,255,0),2)
        cv2.imshow("Faces",frame)
        print(a)
        if (a>29):
            break
    camera.release()
    cv2.destroyAllWindows()

def tambah_data_wajah():
    global  data_nama
    data_nama = StringVar()
    button_input = Button(window, text = 'Input Nama',command=input_nama)
    button_input.grid(column=1,row=1)
    entry_name = Entry(window, textvariable=data_nama)
    entry_name.grid(column=0,row=1)
    return data_nama

def training():
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    detector = pathlib.Path(cv2.__file__).parent.absolute()/"data/haarcascade_frontalface_default.xml"
    detector = cv2.CascadeClassifier(str(detector))
    def getImagesWithLabels(path):
        imagePaths=[os.path.join(path,f) for f in os.listdir(path) if f not in ['.DS_Store', '~$aining.xml']]
        faceSamples=[]
        Ids=[]
        for imagePath in imagePaths:
            pilImage=Image.open(imagePath).convert('L')
            imageNp=np.array(pilImage,'uint8')
            Id=int(os.path.split(imagePath)[-1].split(".")[2])
            faces=detector.detectMultiScale(imageNp)
            for (x,y,w,h) in faces:
                faceSamples.append(imageNp[y:y+h,x:x+w])
                Ids.append(Id)
        return faceSamples, Ids

    file_path = "Deteksi Wajah/Dataset/training.xml"

    # Check if the file exists before deleting
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"File '{file_path}' deleted successfully.")
    else:
        print(f"File '{file_path}' does not exist.")

    faces, Ids = getImagesWithLabels('Deteksi Wajah/Dataset')
    print(faces,Ids)
    recognizer.train(faces, np.array(Ids))
    recognizer.save('Deteksi Wajah/Dataset/training.xml')
    
def deteksi_wajah():
    import pathlib
    import os
    from PIL import Image
    import cv2
    cascade_path = pathlib.Path(cv2.__file__).parent.absolute()/"data/haarcascade_frontalface_default.xml"
    clf = cv2.CascadeClassifier(str(cascade_path))

    camera = cv2.VideoCapture(0)
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read('Deteksi Wajah/Dataset/training.xml')
    while True:
        check, frame = camera.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = clf.detectMultiScale(
        gray,
        scaleFactor = 1.1,
        minNeighbors = 10,
        minSize=(30,30),
        flags = cv2.CASCADE_SCALE_IMAGE)
        data = pd.read_excel('Deteksi Wajah/Tabel_Deteksi_Wajah.xlsx',header=0)
        for (x,y,width,height) in faces:
            cv2.rectangle(frame, (x,y),(x+width, y+height),(255,255,0),2)
            id, conf = recognizer.predict(gray[y:y+height,x:x+width])
            id_nama = data.loc[data['Label']==id,'Nama'].values[0]
            cv2.putText(frame,id_nama,(x+40,y-10),cv2.FONT_HERSHEY_DUPLEX,1,(0,255,0))
        cv2.imshow("Faces",frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):  # Press 'q' to break
            break
    camera.release()
    cv2.destroyAllWindows()

from tkinter import *

window = Tk()
window.title('Program Deteksi Wajah')
button_tambah = Button(window, text='Tambah Identitas Wajah Baru')
button_tambah.config(command=tambah_data_wajah)
button_tambah.grid(column=0,row=0)
button_training = Button(window, text='Proses')
button_training.config(command=training)
button_training.grid(column=0,row=2)
button_deteksi = Button(window, text='Mulai Video')
button_deteksi.config(command=deteksi_wajah)
button_deteksi.grid(column=0,row=3)
window.mainloop()